In [ ]:
print("all ok")

all ok


In [ ]:
from langchain_groq import ChatGroq

In [ ]:
from dotenv import load_dotenv

In [ ]:
load_dotenv()

True

In [ ]:
llm=ChatGroq(model="deepseek-r1-distill-llama-70b")

In [ ]:
print(llm.invoke("What is the capital of France?").context)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [ ]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

### 1. Data Ingestion

In [ ]:
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
import os

In [ ]:
file_path= os.path.join(os.getcwd(), "data", "sample.pdf")

In [ ]:
loader = PyPDFLoader(file_path)

In [ ]:
documents = loader.load()

In [ ]:
len(documents)

13

In [ ]:
## this is an experimental value. There is no specific chunk size.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len,
)

In [ ]:
docs = text_splitter.split_documents(documents)

In [ ]:
len(docs)

95

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
vector_store = FAISS.from_documents(docs, embedding_model)

In [ ]:
embedding_model.embed_documents(docs[0].page_content)

In [ ]:
len(embedding_model.embed_documents(docs[0].page_content)[0])

In [ ]:
relevant_doc = vector_store.similarity_search("What is the research question here?", k=3)

In [ ]:
relevant_doc[0].page_content

In [ ]:
relevant_doc[1].page_content

In [ ]:
relevant_doc[2].page_content

In [ ]:
#retriever = vector_store.as_retriever()

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})    

In [ ]:
retriever.invoke("What is the research question here?", k=3)

In [ ]:
prompt_template = """
    Answer the question based on the context provided below.
    If the context does not contain sufficient information, respond with:
    "I do not have enough information about this."

    Context: {context}

    Question: {question}

    Answer:"""

In [ ]:
from langchain_core.prompts import PromptTemplate


In [ ]:
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)

In [ ]:
prompt

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser = StrOutputParser()

In [ ]:
rag_chain = prompt | llm | parser

In [ ]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])


In [ ]:
from langchain_core.runnables import RunnablePassthrough


In [ ]:
rag_chain = (
    {
        "context": retriever | format_docs, "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
rag_chain.invoke("what is section 3 about")